In [2]:
import matplotlib.pyplot as plt
import nibabel as nib
import numpy as np
import pandas as pd
import seaborn as sns
# import torch
# from torch import Tensor
# from tqdm import tqdm
import os
import io
import glob
import math
import sklearn
import nilearn as nl
from sklearn.manifold import TSNE
from sklearn.model_selection import train_test_split
from sklearn.svm import SVR
from sklearn.preprocessing import StandardScaler
from sklearn.linear_model import LinearRegression
from sklearn.metrics import mean_absolute_error, mean_squared_error 
from sklearn.svm import LinearSVR
from sklearn_rvm import EMRVR

In [79]:
# Reading the reduced data matrix and the subject metadata

# dataset_name = "reduced_PCA_openbhb_train_dataset"
# dataset_name = "openbhb_train_roi_dataset"
dataset_name = "CAT12_VBM_features_FreeSurfer_Xhemi_features_Quasi_Raw_features_JL_Transform_openbhb_train_dataset"

# train_set_name = "reduced_kernel_trick_openbhb_train_set_500_components"
# test_set_name = "reduced_kernel_trick_openbhb_val_set_500_components"

# # MRI_data = np.array(pd.read_csv("/media/dataanalyticlab/Drive2/MANSOOR/Dataset/OpenBHB/" + dataset_name + ".csv", header=None, usecols=np.arange(1,1797)))
X = np.load("/media/dataanalyticlab/Drive2/MANSOOR/Dataset/OpenBHB/Reduced_matrices_openbhb/" + dataset_name + ".npy")
# X_test = np.array(pd.read_csv("/media/dataanalyticlab/Drive2/MANSOOR/Dataset/OpenBHB/Reduced_matrices_openbhb/" + test_set_name + ".csv", usecols=np.arange(1,501)))


# train_metadata = np.array(pd.read_csv('/media/dataanalyticlab/Drive2/MANSOOR/Dataset/openbhb_sub_metadata_k-means_cluster.csv'))
# test_metadata = np.array(pd.read_csv('/media/dataanalyticlab/Drive2/MANSOOR/Dataset/OpenBHB/Validation_set/participants.tsv', delimiter="\t"))
# sub_metadata = np.array(pd.read_csv('/media/dataanalyticlab/Drive2/MANSOOR/Dataset/openbhb_sub_metadata_k-means_cluster.csv'))

# Y_train = train_metadata[:,2].reshape(-1,1)
# Y_test = test_metadata[:756,3].reshape(-1,1)

In [68]:
dataset = []
Model = []
MAE = []
MSE = []
RMSE = []
R_sq = []

In [80]:
# Splitting the dataset into the Training set and Test set

Y = sub_metadata[:, 2]

X_train, X_test, Y_train, Y_test = train_test_split(X, Y, test_size = 0.2, random_state = 0)

Y = Y.reshape(-1,1)

In [31]:
# Feature Scaling

sc_X = StandardScaler()
sc_Y = StandardScaler()
X = sc_X.fit_transform(X)
Y = sc_Y.fit_transform(Y)

In [81]:
LR(X_train, Y_train, X_test, Y_test)

In [82]:
SVR(X_train, Y_train, X_test, Y_test)

/usr/local/lib/python3.10/dist-packages/sklearn/svm/_base.py:1244: ConvergenceWarning: Liblinear failed to converge, increase the number of iterations.
  warnings.warn(


In [83]:
RVR(X_train, Y_train, X_test, Y_test)

/usr/local/lib/python3.10/dist-packages/sklearn_rvm/em_rvm.py:324: UserWarning: Hessian not positive definite
  warnings.warn("Hessian not positive definite")


Linear Regression Implementation

In [67]:
def LR(X_train, Y_train, X_test, Y_test):

    lm = LinearRegression()

    lm.fit(X_train, Y_train)

    # print("Intercept: {:,.3f}".format(lm.intercept_))
    # print("Coefficient: {:,.3f}".format(lm.coef_[1]))

    dataset.append(dataset_name)
    Model.append("Linear Regression")
    Y_pred = lm.predict(X_test)

    MAE.append(round(mean_absolute_error(Y_test, Y_pred), 2))

    mse = round(mean_squared_error(Y_test, Y_pred), 2)
    MSE.append(mse)
    RMSE.append(round(math.sqrt(mse),2))
    R_2 = round(sklearn.metrics.r2_score(Y_test, Y_pred),2)
    R_sq.append(R_2)



Support Vector Regression (SVR) Implementation

In [11]:
def svr_results(y_test, X_test, fitted_svr_model):
    
    print("C: {}".format(fitted_svr_model.C))
    print("Epsilon: {}".format(fitted_svr_model.epsilon))
    
    # print("Intercept: {:,.3f}".format(fitted_svr_model.intercept_[0]))
    # print("Coefficient: {:,.3f}".format(fitted_svr_model.coef_[0]))
    
    mae = mean_absolute_error(Y_test, fitted_svr_model.predict(X_test))
    print("MAE = ", mae)
    mse = mean_squared_error(Y_test, fitted_svr_model.predict(X_test))
    rmse = math.sqrt(mse)
    print("RMSE = ", rmse)
    
    # Plot outputs
    # plt.figure(figsize=(10,7))
    # # plt.scatter(x=df['rooms'], y=df['cmedv'])
    # plt.plot(X_test, fitted_svr_model.predict(X_test), color='red')
    # plt.plot(X_test, fitted_svr_model.predict(X_test)+eps, color='black')
    # # plt.plot(X_test, fitted_svr_model.predict(X_test)-eps, color='black')
    # plt.xlabel('# of Rooms')
    # plt.ylabel('House Price (Thousands of Dollars)')
    # plt.title('SVR Prediction')
    # plt.show()

In [70]:
def SVR(X_train, Y_train, X_test, Y_test):
    eps = 5
    svr = LinearSVR(epsilon=eps, C=0.14, fit_intercept=True)
    svr.fit(X_train, Y_train)

    # storing the results
    dataset.append(dataset_name)
    Model.append("Linear Support Vector Regression")
    Y_pred = svr.predict(X_test)

    MAE.append(round(mean_absolute_error(Y_test, Y_pred), 2))

    mse = round(mean_squared_error(Y_test, Y_pred), 2)
    MSE.append(mse)
    RMSE.append(round(math.sqrt(mse),2))
    R_2 = round(sklearn.metrics.r2_score(Y_test, Y_pred),2)
    R_sq.append(R_2)



Revelance Vector Regression (RVR) Implementation

In [71]:
def RVR(X_train, Y_train, X_test, Y_test):
    
    clf = EMRVR(kernel='linear')
    clf.fit(X_train, Y_train)

    dataset.append(dataset_name)
    Model.append("Relevance Vector Regression")
    Y_pred = clf.predict(X_test)

    MAE.append(round(mean_absolute_error(Y_test, Y_pred), 2))

    mse = round(mean_squared_error(Y_test, Y_pred), 2)
    MSE.append(mse)
    RMSE.append(round(math.sqrt(mse),2))
    R_2 = round(sklearn.metrics.r2_score(Y_test, Y_pred),2)
    R_sq.append(R_2)


Model Evaluation Metrics

In [86]:

dataset_s = pd.Series(dataset)
Model_s = pd.Series(Model)
MAE_s = pd.Series(MAE)
RMSE_s = pd.Series(RMSE)
R_sq_s = pd.Series(R_sq)
MSE_s = pd.Series(MSE)


evaluation_metrics = pd.DataFrame({"Dataset": dataset_s, "Model": Model_s, "RMSE": RMSE_s, "MSE": MSE_s, "MAE": MAE_s,  "R_sq": R_sq_s})

In [87]:
evaluation_metrics

,Dataset,Model,RMSE,MSE,MAE,R_sq
0,CAT12_VBM_features_Quasi_Raw_features_JL_Trans...,Linear Regression,2354.27,5542568.50,323.12,-25371.09
1,CAT12_VBM_features_Quasi_Raw_features_JL_Trans...,Linear Support Vector Regression,2798.54,7831804.61,400.38,-35850.48
2,CAT12_VBM_features_Quasi_Raw_features_JL_Trans...,Relevance Vector Regression,318.37,101358.62,49.45,-462.99
3,FreeSurfer_Xhemi_features_JL_Transform_openbhb...,Linear Regression,10.36,107.24,8.17,0.51
4,FreeSurfer_Xhemi_features_JL_Transform_openbhb...,Linear Support Vector Regression,9.25,85.63,7.23,0.61
5,FreeSurfer_Xhemi_features_JL_Transform_openbhb...,Relevance Vector Regression,9.18,84.25,6.94,0.61
6,CAT12_VBM_features_FreeSurfer_Xhemi_features_Q...,Linear Regression,2802.16,7852087.96,375.52,-35943.33
7,CAT12_VBM_features_FreeSurfer_Xhemi_features_Q...,Linear Support Vector Regression,2356.72,5554144.85,319.92,-25424.09
8,CAT12_VBM_features_FreeSurfer_Xhemi_features_Q...,Relevance Vector Regression,216.86,47026.50,34.21,-214.27


In [59]:
evaluation_metrics.to_csv("/media/dataanalyticlab/Drive2/MANSOOR/Dataset/OpenBHB/Results/Regression/Regression_results_pca_with_subsets_and_combined_features_train.csv")

Vary C (find the best value for C)

In [ ]:
test_mae_list = []
perc_within_eps_list = []

eps = 5
c_space = np.linspace(0.01, 10)

for c in c_space:
    varied_svr = LinearSVR(epsilon=eps, C=c, fit_intercept=True, max_iter=10000)
    
    varied_svr.fit(X_train, Y_train)
    
    test_mae = mean_absolute_error(Y_test, varied_svr.predict(X_test))
    test_mae_list.append(test_mae)
    
    perc_within_eps = 100*np.sum(abs(Y_test-varied_svr.predict(X_test)) <= eps) / len(Y_test)
    perc_within_eps_list.append(perc_within_eps)